In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import socket
import time 
from flask import Flask

In [2]:
# Создаем объект приложения Dash
app = dash.Dash(__name__)

# создаем подложку для графиков 
plots = []

# Определяем макет (layout) приложения
app.layout = html.Div(
    [
        html.H1("Можно сделать шапку"),

        # Выводим график с использованием компонента dcc.Graph для отображения графических данных
        dcc.Graph(id="polar-plot"),

        # Устанавливаем интервал обновления в 1 секунду
        dcc.Interval(
            id="interval-component",
            interval=1000,  # Обновление каждую секунду
            n_intervals=0,
        ),
    ]
)


# Определяем функцию-колбэк для обновления графика
@app.callback(
    Output("polar-plot", "figure"),
    [Input("interval-component", "n_intervals")]
)
# Генерируем значения из второго кода для графика
def update_polar_plot(n):
    MAX_CONNECTIONS = 1
    address_to_server = ('192.168.54.123', 8001)
    clients = [socket.socket(socket.AF_INET, socket.SOCK_STREAM) for i in range(MAX_CONNECTIONS)]
    
    try:
        for client in clients:
            client.connect(address_to_server)

        for i in range(MAX_CONNECTIONS):
            clients[i].send(bytes(f"\x01\x03\x00\x00\x00\x07\x04\x08{i}",
                                  encoding='UTF-8'))
            
    except TimeoutError:
        print('TimeoutError при получении данных')
        
    for client in clients:
        try:
            input_data = client.recv(128)
        except socket.timeout:
            print("Timeout occurred while receiving data.")
            continue # или сделать что-то другое, чтобы программа не падала
        
        hex_data = input_data.hex()  
        SokolAnswer = hex_data
        value_01 = SokolAnswer[0:2]
        value_03 = SokolAnswer[2:4]
        value_0e = SokolAnswer[4:6]
        Device_Type = SokolAnswer[6:10]
        WindSpeedMult10 = int(SokolAnswer[10:14],16)
        WindSpeedList = [WindSpeedMult10/10] 
        WindGustMult10_10min = int(SokolAnswer[14:18],16)
        WindSpeedMult10_10min = int(SokolAnswer[18:22],16)
        WindSpeedMult10_1min = int(SokolAnswer[22:26],16)
        WindDir_1min = int(SokolAnswer[26:30],16)
        WindDirList = [WindDir_1min]
        WindDir_10min = int(SokolAnswer[30:34],16)
        print(WindSpeedList, WindDirList)

        # Создаем объект графика в полярной системе координат
        fig = go.Figure(go.Barpolar(
                    r=WindSpeedList,
                    theta=WindDirList,
                    width=8,
                    base=0,
                    offset=-4,
                    marker_line_width = 0,
                    hovertext=["Скорость ветра: {:.1f}, БББ: {:.0f}".format(WindSpeedList[0], WindDirList[0])],
                    hovertemplate="Скорость ветра: %{WindSpeedList:.1f} м/с <br> БББ: %{WindDirList:.0f}°<extra></extra>",
                    hoverlabel=dict(font=dict(size=14, color='white')),
                    showlegend=True,
                    name="Скорость ветра: {:.1f} м/с<br>Направление: {:.0f}°".format(WindSpeedList[0], WindDirList[0])))
        fig.update_layout(title='Устройство №: ', width=800, height=520, 
                      polar=dict(
                                 angularaxis=dict(direction="clockwise",
                                                  tickvals=np.arange(0, 360, 22.5),
                                                  ticktext=["С", "ССВ", "СВ", "ВСВ",
                                                            "В", "ВЮВ", "ЮВ", "ЮЮВ",
                                                            "Ю", "ЗЗЮ", "ЗЮ", "ЗЮЗ",
                                                            "З", "ЗСЗ", "СЗ", "ССЗ"],
                                                    ),
                                 radialaxis=dict(range=[0, 8])),
                                 legend=dict(x=0.31, y=-0.3,
                                              traceorder='normal',
                                              font=dict(size=16),
                                              bgcolor='rgba(0,0,0,0)',
                                              bordercolor='rgba(0,0,0,0)'))
                                             
                                                             
        return fig
if __name__ == "__main__":
    # Запускаем приложение
    app.run_server(debug=False, port = 8090)

[0.0] [191]
[0.0] [191]
[0.0] [191]
[0.0] [191]
[0.0] [191]
Timeout occurred while receiving data.


[2023-07-18 14:19:02,166] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\Guest\AppData\Roaming\Python\Python310\site-packages\dash\dash.py", line 1265, in dispatch
    ctx.run(
  File "C:\Users\Guest\AppData\Roaming\Python\Python310\site-packages\dash\_callback.py", line 450, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callb

TimeoutError при получении данных


[2023-07-18 14:19:05,157] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\Guest\AppData\Roaming\Python\Python310\site-packages\dash\dash.py", line 1265, in dispatch
    ctx.run(
  File "C:\Users\Guest\AppData\Roaming\Python\Python310\site-packages\dash\_callback.py", line 450, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callb

TimeoutError при получении данных
Timeout occurred while receiving data.
[3.0] [191]
[3.0] [191]
Timeout occurred while receiving data.
[0.5] [191]
[0.5] [191]
[0.5] [191]
[0.5] [191]
[0.5] [191]
[0.5] [191]
[0.5] [191]
[0.5] [191]
[0.5] [191]


[2023-07-18 14:19:30,263] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\Guest\AppData\Roaming\Python\Python310\site-packages\dash\dash.py", line 1265, in dispatch
    ctx.run(
  File "C:\Users\Guest\AppData\Roaming\Python\Python310\site-packages\dash\_callback.py", line 450, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callb

TimeoutError при получении данных
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] [190]
[0.0] 

In [4]:
from flask import Flask

In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import random
import socket
import re
import time 
import socket
import re
import time 

# Определение сервера
MAX_CONNECTIONS = 1
address_to_server = ('192.168.54.123', 8001)

clients = [socket.socket(socket.AF_INET, socket.SOCK_STREAM) for i in range(MAX_CONNECTIONS)]
for client in clients:
    client.connect(address_to_server)

for i in range(MAX_CONNECTIONS):
    clients[i].send(bytes(f"\x01\x03\x00\x00\x00\x07\x04\x08{i}", encoding='UTF-8'))

# Создаем объект приложения Dash
app = dash.Dash(__name__)

# создаем подложку для графиков 
plots = []

# Определяем макет (layout) приложения
app.layout = html.Div(
    [
        html.H1("Динамическая Колонка № 1 "),

        # Выводим график с использованием компонента dcc.Graph для отображения графических данных
        dcc.Graph(id="polar-plot"),

        # Устанавливаем интервал обновления в 1 секунду
        dcc.Interval(
            id="interval-component",
            interval=500,  # Обновление каждую 0.5 секунду
            n_intervals=0,
        ),
    ]
)


# Определяем функцию-колбэк для обновления графика
@app.callback(
    Output("polar-plot", "figure"),
    [Input("interval-component", "n_intervals")]
               )



# Генерируем случайные значения для графика
def update_polar_plot(n):
    
    
    r = [random.uniform(0, 20)]  # Радиусы
    theta = [random.uniform(0, 360)]  # Углы
    
    
    # Создаем объект графика в полярной системе координат
    fig = go.Figure(go.Barpolar(
                r=r,
                theta=theta,
                width=8,
                base=0,
                offset=-4,
                marker_line_width=0,
                hovertext=["Скорость ветра: {:.1f}, БББ: {:.0f}".format(r[0], theta[0])],
                hovertemplate="Скорость ветра: %{r:.1f} м/с <br> БББ: %{theta:.0f}°<extra></extra>",
                hoverlabel=dict(font=dict(size=14, color='white')),
                showlegend=True,
                name="Скорость ветра: {:.1f} м/с<br>Направление: {:.0f}°".format(r[0], theta[0])))
    
    fig.update_layout(title='Устройство №: ', width=800, height=520, 
                      polar=dict(
                                 angularaxis=dict(direction="clockwise",
                                                  tickvals=np.arange(0, 360, 22.5),
                                                  ticktext=["С", "ССВ", "СВ", "ВСВ",
                                                            "В", "ВЮВ", "ЮВ", "ЮЮВ",
                                                            "Ю", "ЗЗЮ", "ЗЮ", "ЗЮЗ",
                                                            "З", "ЗСЗ", "СЗ", "ССЗ"],
                                                    ),
                                 radialaxis=dict(range=[0, 20])),
                                  legend=dict(x=0.31, y=-0.3, traceorder='normal', font=dict(size=16), bgcolor='rgba(0,0,0,0)', bordercolor='rgba(0,0,0,0)'))
                                             
                                            
                  
    return fig


if __name__ == "__main__":
    # Запускаем приложение
    app.run_server(debug=False, port = 8060)


PermissionError: [WinError 10013] Сделана попытка доступа к сокету методом, запрещенным правами доступа